# Length-Prefixed Protobuf Messages
> It's already built-in to gogo/protobuf

- toc: true 
- badges: true
- comments: true
- categories: [go, protobuf]
- image: /images/copied_from_nb/images/tlv.png

![Length-Prefixed Message](images/tlv.png)

[Protocol buffers](https://developers.google.com/protocol-buffers) are popular, 
especially indirectly via [gRPC](https://grpc.io/). If you are using `golang`,
you may use the official [`golang/protobuf`](https://github.com/golang/protobuf)
package. But, there are other implementations. In particular, 
[`gogo/protobuf`](https://github.com/gogo/protobuf) does protocol buffers in 
go *without* reflection, and with smarter allocations to reduce gc pressure. 
The [result is better performance](https://github.com/alecthomas/go_serialization_benchmarks#results).

The [README](https://github.com/gogo/protobuf) does a good job of getting you started. However, 
if you are using `protobuf` without gRPC, you'll quickly find yourself in need of a serialization
format for delimiting messages. The easiest way to do this is with length-prefixing. What I failed
to realize for far too long is that `gogo/protobuf` tools for this already.

Looking at `github.com/gogo/protobuf/io`, there is

```go
func NewDelimitedReader(r io.Reader, maxSize int) ReadCloser {
	var closer io.Closer
	if c, ok := r.(io.Closer); ok {
		closer = c
	}
	return &varintReader{bufio.NewReader(r), nil, maxSize, closer}
}
```

and,

```go
func NewDelimitedWriter(w io.Writer) WriteCloser {
	return &varintWriter{w, make([]byte, binary.MaxVarintLen64), nil}
}
```

The reader has a `ReadMsg` function,

```go
func (this *varintReader) ReadMsg(msg proto.Message) error {
```

and the writer has a `WriteMsg` function,

```go
func (this *varintWriter) WriteMsg(msg proto.Message) (err error)
```

Both these methods use `encoding/binary`'s [`Uvarint`](https://golang.org/pkg/encoding/binary/#ReadUvarint), allowing for very large messages — no need to roll your own.

Depending on your protocol/serialization needs, it may be useful to use a [TLV encoding](https://en.wikipedia.org/wiki/Type-length-value). Given `DelimitedReader` and `DelimitedWriter`,
the easiest way to do this is to write the type as a byte before each length-delimited message
is sent. I generally define mine in the [protocol file as an enum](https://developers.google.com/protocol-buffers/docs/proto3#enum), since it's self documenting.

Anyways, nothing in this post is new or interesting. It's just that I didn't realize it was already built 
into a dependency I was using anyway, so hopefully I'll save you a stack-overflow trip.